In [272]:
# Important libaries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

Speed up the work by working on the stratified sample for now

In [273]:
# Read original CSV data
#vehicledf = pd.read_csv("W:/DS/Programming/Data/CMUvehicle.csv")

In [274]:
#vehicledf.head()

In [275]:
#strat_sample = vehicledf.groupby('type').apply(lambda x: x.sample(frac=0.05, random_state=42))

In [276]:
#strat_sample.to_csv('stratified_sample_CMUvehicle.csv')

# The work

In [277]:
vehicledf = pd.read_csv("/kaggle/input/vehicle/stratified_sample_CMUvehicle.csv")

In [278]:
vehicledf.head()

,type,Unnamed: 1,id,camera_id,plate_number,type.1,source_datetime,Year,is_registered,direction,gate_id,gate_name
0,car,18883147,243905453,1507,มช917,car,3/12/2023 8:56:55 AM,2023,False,i,32,แยกโรงอาหารใหม่
1,car,33748683,259632945,1803,9กบ2197,car,6/7/2023 9:45:54 AM,2023,False,o,34,แยกบริหาร
2,car,18027612,243010725,20010,มช1029,car,3/8/2023 7:20:18 PM,2023,True,i,21,ลานจอดรถหน้ามอ
3,car,33832189,259723660,659790113,งษ9960,car,6/7/2023 4:44:30 PM,2023,False,i,19,ปตท.ใหม่
4,car,31529547,257238203,1203,มช030,car,5/24/2023 12:46:08 PM,2023,False,o,23,วงเวียน SCB


In [279]:
vehicledf[vehicledf['Year'] != 2023]

,type,Unnamed: 1,id,camera_id,plate_number,type.1,source_datetime,Year,is_registered,direction,gate_id,gate_name


Check if camera_id is the same as gate_id by checking for one-to-one relationship:

In [280]:
def isonetoone(df,col1,col2):
    # A short code to check for a one-to-one relationship
    # group rows by camera_id, then find if numbers of unique values in gate_id is one for all groups
    # True means the two columns have a one-to-one relationship
    return df.groupby(col1)[col2].apply(lambda x: x.nunique() == 1).all()

isonetoone(vehicledf,'camera_id','gate_id')

True

The two features have a one-to-one relationship, proceed to drop one of them. In this case, camera_id is drop since is less interpretable. Next, check for one-to-one relationship between gate_id and gate_name:

In [281]:
isonetoone(vehicledf,'gate_id','gate_name')

True

This mean that one can use gate_id to represent both camera_id and gate_name. However, I build a dictionary between gate_id and gate name for further analysis.

In [282]:
GATENAME = pd.Series(vehicledf['gate_name'].values,index=vehicledf['gate_id']).to_dict()

In [283]:
GATENAME

{32: 'แยกโรงอาหารใหม่',
 34: 'แยกบริหาร',
 21: 'ลานจอดรถหน้ามอ',
 19: 'ปตท.ใหม่',
 23: 'วงเวียน SCB',
 33: 'วงเวียนหอนาฬิกา',
 27: 'วงเวียนสนามเทนนิส',
 3: 'คันคลองชลประทาน',
 4: 'เกษตรศาสตร์',
 1: 'หน้ามหาวิทยาลัย',
 24: 'วงเวียนอ่างตาดชมพู',
 36: 'วิศวกรรมศาสตร์ใหม่',
 30: 'แยก อมช',
 5: 'วิศวกรรมศาสตร์',
 26: 'แยกประตูไผ่ล้อม',
 38: 'ลานจอดรถ  S1',
 22: 'แยกอ่างแก้ว',
 25: 'วงเวียนมนุษย์',
 41: 'สวนดอกพาร์ค',
 31: 'ลานจอดรถฝายหิน',
 20: 'แยกตึกอธิการบดี',
 2: 'ไผ่ล้อม',
 29: 'ลานจอดรถ อมช',
 18: 'ศึกษาศาสตร์',
 40: 'จอดรถโรงอาหารกลาง',
 42: 'POC-ENG-in',
 37: 'อาคารสำนักงาน 3',
 35: 'ลานจอดรถไร่ฟอร์ด',
 28: 'ลานจอดรถหอ 40 ปี',
 39: 'ข่วงพยอม'}

Drop unnecessary columns for modelling, including:
* id 
* Unnamed: 1 - resultant column of stratified sampling meaning the previous index 
* type.1 - duplication of type)
* Year - Only instances from 2023 is found
* camera_id - has one-to-one relationship to gate_id
* gate_name - has one-to-one relationship to gate_id

In [284]:
vehicledf.drop(columns = ['id','Unnamed: 1','type.1','Year','camera_id','gate_name'], inplace = True)
vehicledf.head()

,type,plate_number,source_datetime,is_registered,direction,gate_id
0,car,มช917,3/12/2023 8:56:55 AM,False,i,32
1,car,9กบ2197,6/7/2023 9:45:54 AM,False,o,34
2,car,มช1029,3/8/2023 7:20:18 PM,True,i,21
3,car,งษ9960,6/7/2023 4:44:30 PM,False,i,19
4,car,มช030,5/24/2023 12:46:08 PM,False,o,23


In [285]:
vehicledf.shape

(1707713, 6)

Keep only instances that are near to CMU entrances:

In [286]:
'''{19: 'ปตท.ใหม่',
 4: 'เกษตรศาสตร์',
 1: 'หน้ามหาวิทยาลัย',
 36: 'วิศวกรรมศาสตร์ใหม่',
 5: 'วิศวกรรมศาสตร์',
 2: 'ไผ่ล้อม',
 18: 'ศึกษาศาสตร์',
 42: 'POC-ENG-in'}
'''
entrance_mask = [19,4,1,36,5,2,18,42]
vehicledf = vehicledf[vehicledf['gate_id'].isin(entrance_mask )]

Convert source_datetime to Pandas datetime. Then, filter out vehicles before 08.30 AM and after 15.10 PM

In [288]:
datetime = pd.to_datetime(vehicledf['source_datetime'],format='mixed')
vehicledf['month'] = datetime.dt.month_name()
vehicledf['day'] = datetime.dt.day
vehicledf['hour'] = datetime.dt.hour
vehicledf['min'] = datetime.dt.minute
vehicledf['sec'] = datetime.dt.second
vehicledf.drop(columns = ['source_datetime'], inplace = True)

Split data by month for further processing:

In [298]:
months = vehicledf['month'].unique()
for m in months:
    vehicledf[vehicledf['month'] ==  m].to_csv('vehicle_'+m+'.csv')

# Encoding

In [ ]:
one_hot_type = pd.get_dummies(vehicledf['type'])
#one_hot_plate_number = pd.get_dummies(vehicledf['plate_number'])
#one_hot_is_registered = pd.get_dummies(vehicledf['is_registered'])
#one_hot_direction = pd.get_dummies(vehicledf['direction'])

In [ ]:
vehicledf = vehicledf.drop('type',axis = 1)
vehicledf = vehicledf.join(one_hot_type)

#vehicledf = vehicledf.drop('is_registered',axis = 1)
#vehicledf = vehicledf.join(one_hot_is_registered)

#vehicledf = vehicledf.drop('direction',axis = 1)
#vehicledf = vehicledf.join(one_hot_direction)

# Clustering with Kmeans

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=0, n_init="auto").fit(vehicledf.drop(columns = ['source_datetime','plate_number','is_registered','direction']))

In [ ]:
vehicledf['cluster']= kmeans.labels_.astype('str')

In [ ]:
#sns.pairplot(vehicledf.drop(columns = ['source_datetime','plate_number','is_registered','direction']), hue = 'cluster')
#plt.plot()